In [1]:
import os
import sys
import torch.nn.utils.rnn
from torch.optim import Adam
from torch.utils.data import Dataset
from torchmetrics import Dice
from tqdm import tqdm

sys.path.append(os.path.abspath('./code/'))
from fedlab.utils.functional import get_best_device
from model.resnet import resnet50
from utils.dataloader import EchoDataset, EchoCollator,EchoFrameDataset, EchoFrameCollator


In [2]:
from preprocess.echo_splittting import ECHOSplitter

# spt = ECHOSplitter('/data/zyk/data/dataset/ECHO/preprocessed/client2/metadata.csv',
#                    train_output_file='/data/zyk/data/dataset/ECHO/preprocessed/client2/train.csv',
#                    test_output_file='/data/zyk/data/dataset/ECHO/preprocessed/client2/test.csv',
#                    test_sample_rate=0.2, random_seed=2
#                    )
# spt.run()

In [2]:
ds_train = EchoFrameDataset(meta_name='train',location='client1')

In [3]:
train_dataloader = torch.utils.data.DataLoader(ds_train, batch_size=13, collate_fn=EchoFrameCollator())


In [5]:
for batch in train_dataloader:
    print(batch['labeled']['frames'].shape)
    break

torch.Size([1, 112, 112])


In [4]:

from code.utils.dataloader import EchoFrameCollator

ds_train = EchoFrameDataset(meta_name='train',location='client2')
ds_test = EchoFrameDataset(meta_name='test',location='client2')

epochs = 10
batch_size = 16

train_dataloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, collate_fn=EchoFrameCollator())
ds_test = EchoDataset(meta_name='test',location='client2')

test_dataloader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, collate_fn=EchoCollator())


def evaluate(model, loader, ignore_index=-200):
    metric = Dice(ignore_index=ignore_index, num_classes=4)
    model.eval()
    with torch.no_grad():
        preds = []
        labels = []
        for batch in tqdm(loader, desc='evaluating...'):
            input = batch['videos'].to(model.device)
            for i in range(input.shape[1]):
                optimizer.zero_grad()
                ipt = torch.unsqueeze(input[:, i, ...], dim=1)
                ipt = torch.cat([ipt, ipt, ipt], dim=1)
                ipt = ipt.float() / 255.0
                output = model(ipt)
                label = batch['labels'][:, i, ...]
                pred = output['out'].permute(0, 2, 3, 1).reshape(-1, 4).detach().cpu()
                label = label.reshape(-1)
                preds.append(pred)
                labels.append(label)
        preds = torch.cat(preds, dim=0)
        labels = torch.cat(labels, dim=0)
        # skip labels where it is -200
        preds = preds[labels != ignore_index]
        labels = labels[labels != ignore_index]
        dice = metric(preds, labels)
    model.train()
    return dice


In [4]:
device = get_best_device()
model = resnet50(num_classes=4).to(device)
model.device = device
optimizer = Adam(model.parameters(), lr=1e-3)


pbar = tqdm(total=len(train_dataloader) * epochs)
for epc in range(epochs):
    for batch in train_dataloader:
        input = batch['videos'].to(device)
        loss = 0
        lf = torch.nn.CrossEntropyLoss(ignore_index=-200, reduction='mean')
        for i in range(input.shape[1]):
            optimizer.zero_grad()
            ipt = torch.unsqueeze(input[:, i, ...], dim=1)
            ipt = torch.cat([ipt, ipt, ipt], dim=1)
            ipt = ipt.float() / 255.0
            output = model(ipt)
            label = batch['labels'][:, i, ...]
            pred = output['out'].permute(0, 2, 3, 1).reshape(-1, 4)
            label = label.to(device).reshape(-1)
            ls = lf(pred, label)
            if not ls.isnan():
                loss += ls
        loss.backward()
        optimizer.step()
        pbar.set_postfix(dict(epc=epc, loss=loss.item()))
        pbar.update()
    res = evaluate(model, test_dataloader)
    print(f'epoch {epc} dice: {res.item()}')

  0%|                                                                                                                                              | 0/5020 [00:00<?, ?it/s]/data/stupidtree/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
  5%|█████▍                                                                                                         | 245/5020 [10:53<3:57:30,  2.98s/it, epc=0, loss=0.598]
KeyboardInterrupt



In [3]:
ds_test = EchoDataset(meta_name='test',location='client1')
test_dataloader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, collate_fn=EchoCollator(),shuffle=True)


for batch in test_dataloader:
    input = batch['videos'].to(device)
    for i in range(input.shape[1]):
        optimizer.zero_grad()
        ipt = torch.unsqueeze(input[:, i, ...], dim=1)
        ipt = torch.unsqueeze(input[:, i, ...], dim=1)
        ipt = torch.cat([ipt, ipt, ipt], dim=1)
        ipt = ipt.float() / 255.0
        output = model(ipt)
        label = batch['labels'][:, i, ...]
        pred = output['out']
        # show the image
        import matplotlib.pyplot as plt

        plt.imshow(label[0].detach().cpu().numpy())
        plt.show()
        plt.imshow(pred[0].permute(1,2,0).argmax(-1).detach().cpu().numpy())
        plt.show()
        break
    break

NameError: name 'batch_size' is not defined